# Step 03: Data preparation

In this step we enrich the dataset with some information we can find in the text, like:

* When a person has a special position in the government or in the congress (President of the congress, President of the nation, Minister,...), they usually have the position name written in ALL CAPS and then their name follows it between parens. Like this: `PRESIDENTE DEL GOBIERNO (Mariano Rajoy)`
* There's also a process to enrich the dataset with additional information based on paranthesized text.


In [1]:
import pandas as pd
import re

files = !ls data/debates/*.csv

full = pd.concat([pd.read_csv(f) for f in files])

In [2]:
# constants
title_pat=r'(.*)\((.*)\)'

In [3]:
# find patterns like PRESIDENTE DEL GOBIERNO (Mariano Rajoy)

# extract title, ex: "PRESIDENTE DEL GOBIERNO"
full["Title"]=full["Name"].apply(lambda x: re.match(title_pat, x).groups()[0] if re.match(title_pat, x)!=None else None)
# extract name, ex: "Mariano Rajoy"
full["Name"]=full["Name"].apply(lambda x: re.match(title_pat, x).groups()[1].upper() if re.match(title_pat, x)!=None else x)
full[full.Title.notnull()].head()

,Name,Text,Date,Term,Title
37,BELOKI GUERRA,Propuesta de transacción. Extender las relacio...,20080527,IX,SECRETARIO
89,SOLBES MIRA,"Señor presidente, señorías, comparezco de nuev...",20080626,IX,VICEPRESIDENTE SEGUNDO DEL GOBIERNO Y MINISTRO...
102,CUNILLERA I MESTRES,"Muchas gracias, señor Azpiazu. Tiene la palabr...",20080626,IX,VICEPRESIDENTA
104,CUNILLERA I MESTRES,"Señor Sánchez i Llibre, por favor",20080626,IX,VICEPRESIDENTA
106,CUNILLERA I MESTRES,"Gracias a usted, señor Sánchez. Tiene ahora la...",20080626,IX,VICEPRESIDENTA


In [4]:
# find paranthesized values and sort them by occurence
parenthesized_pat=r'(\(.+?\))'
# can also use negative matching instead of non-greedy parenthesized_pat=r'.*(\([^\)]+\))'
l=full["Text"].apply(lambda t: ",".join(re.findall(parenthesized_pat,str(t))))
pd.DataFrame({'test':l}).groupby(['test']).size().reset_index().sort_values(0,ascending=False).head(20)

,test,0
0,,229854
17132,(Pausa.),20707
634,(Aplausos),14816
16867,(Pausa),8988
6392,(Convergència i Unió),5055
17483,(Pausa.-Una trabajadora del servicio de limpie...,4559
20530,(Rumores.),3129
19743,(Rumores),2426
7112,(EAJ-PNV),2181
682,"(Aplausos),(Aplausos)",2020


In [5]:
parenthesized_pat=r'\(.+?\)'
unclosed_parenthesized_pat=r'(\(.+?\.)'
m=full["Text"].str.replace(parenthesized_pat,'',regex=True).str.strip()
l=m.apply(lambda t: ",".join([par+")" for par in re.findall(unclosed_parenthesized_pat,str(t))]))
#pd.DataFrame({'test':l}).groupby(['test']).size().reset_index().sort_values(0,ascending=False).head(20)
tdf=pd.DataFrame({'m':l})
tdf["m"].apply(lambda m: m.replace(".)",")") ).iloc[6065]



'(ANTES DENOMINADO PROYECTO DE LEY ORGÁNICA DE MODIFICACIÓN DE LA LEY ORGÁNICA 6/1985, DE 1 DE JULIO, DEL PODER JUDICIAL Y COMPLEMENTARIA A LA LEY PARA LA EJECUCIÓN EN LA UNIÓN EUROPEA DE RESOLUCIONES QUE IMPONGAN SANCIONES PECUNIARIAS),(Número de expediente 121/000003)'

In [6]:
t="this (is a unclosed. or more (things. whatever"
[t+")" for t in re.findall(unclosed_parenthesized_pat,str(t))]

['(is a unclosed.)', '(things.)']

In [7]:
parenthesized_pat=r'\(.+?\)'
unclosed_parenthesized_pat=r'(\(.+?\.)'

full["infos"]=full["Text"].apply(lambda t: ",".join(re.findall(parenthesized_pat,str(t))).strip())
# now remove parenthesized sentences
full["Text"]=full["Text"].str.replace(parenthesized_pat,'',regex=True).str.strip()
full["infos2"]=full["Text"].apply( lambda t: ",".join([par+")" for par in re.findall(unclosed_parenthesized_pat,str(t))]))
full["Text"]=full["Text"].str.replace(unclosed_parenthesized_pat,'',regex=True).str.strip()
full["infos"]=full[["infos","infos2"]].apply( lambda row: row["infos"] if len(row["infos2"])==0 else row["infos2"] if len(row["infos"])==0 else ",".join([row["infos"],row["infos2"]]),axis=1)
full["infos"]=full["infos"].str.replace(".)",")",regex=False)
del full["infos2"]
full

,Name,Text,Date,Term,Title,infos
0,PRESIDENTE,"Señorías, se inicia la sesión. Tenemos como as...",20080527,IX,None,
1,SECO REVILLA,"Sí, prometo.",20080527,IX,None,(Aplausos)
2,PRESIDENTE,Don Óscar Seco Revilla ha adquirido la condici...,20080527,IX,None,(Número de expediente 162/000025)
3,PRESIDENTE,Pasamos a examinar el punto I del orden del dí...,20080527,IX,None,
4,LLAMAZARES TRIGO,"Gracias, señor presidente. Señorías, con volun...",20080527,IX,None,
...,...,...,...,...,...,...
41341,PRESIDENTA,"En consecuencia, queda aprobado el dictamen. -...",20221103,XIV,None,(Número de expediente 110/000107)
41342,PRESIDENTA,Acuerdo entre el Gobierno del Reino de España ...,20221103,XIV,None,(Pausa)
41343,PRESIDENTA,"En consecuencia, no queda aprobado el dictamen...",20221103,XIV,None,"(VOTACIÓN),(PROCEDENTE DEL REAL DECRETO-LEY 16..."
41344,PRESIDENTA,Votación de la solicitud de avocación por el P...,20221103,XIV,None,(Pausa)


In [8]:
full[(full["infos"].str.len()>0) & (full["Title"].str.len()>0)].head(20)

,Name,Text,Date,Term,Title,infos
89,SOLBES MIRA,"Señor presidente, señorías, comparezco de nuev...",20080626,IX,VICEPRESIDENTE SEGUNDO DEL GOBIERNO Y MINISTRO...,(Aplausos)
102,CUNILLERA I MESTRES,"Muchas gracias, señor Azpiazu. Tiene la palabr...",20080626,IX,VICEPRESIDENTA,(Convergència i Unió)
127,MORATINOS CUYAUBÉ,"Gracias, señor presidente. Señorías, quiero sa...",20080626,IX,MINISTRO DE ASUNTOS EXTERIORES Y DE COOPERACIÓN,(Aplausos)
142,CUNILLERA I MESTRES,Gracias a usted. Por el Grupo Parlamentario Va...,20080626,IX,VICEPRESIDENTA,(EAJ-PNV)
153,MORATINOS CUYAUBÉ,"Gracias, señor presidente. Señorías, interveng...",20080626,IX,MINISTRO DE ASUNTOS EXTERIORES Y DE COOPERACIÓN,(Aplausos)
247,CUNILLERA I MESTRES,"Gracias, señor Beloki. Tiene la palabra, por e...",20080610,IX,VICEPRESIDENTA,"(Convergència i Unió),(El señor presidente ocu..."
299,RODRÍGUEZ ZAPATERO,"Muchas gracias, señor presidente. Señor Erkore...",20080604,IX,PRESIDENTE DEL GOBIERNO,"(Aplausos),(Aplausos)"
303,RODRÍGUEZ ZAPATERO,"Muchas gracias, señor presidente. Señor Erkore...",20080604,IX,PRESIDENTE DEL GOBIERNO,(Aplausos)
308,RODRÍGUEZ ZAPATERO,"Gracias, señor presidente. Señor Duran, compar...",20080604,IX,PRESIDENTE DEL GOBIERNO,(Aplausos)
312,RODRÍGUEZ ZAPATERO,"Muchas gracias, señor presidente. Señor Duran,...",20080604,IX,PRESIDENTE DEL GOBIERNO,(Aplausos)


In [9]:
full.to_parquet('../debates.parquet', index=False)

In [10]:
full[full['Name'].str.contains('EZ ZAPATERO')].to_csv('../zapatero.csv', index=False)
full[full['Name'].str.contains('RAJOY')].to_csv('../rajoy.csv',index=False)

In [11]:
full.to_csv('../debates.csv', index=False)

In [12]:
full[full['Name'].str.contains('RAJOY')].iloc[1]["Text"]

'Señor presidente, hace siete días le pregunté aquí sobre la situación de la economía española y las medidas que usted iba a tomar. No me respondió gran cosa, salvo que estábamos preparados mejor que nadie. Yo le voy a dar algunos datos que hemos conocido esta semana, esto es, desde el miércoles pasado: el IPC está en el 4,7 es decir 0,5 puntos más que en abril; el diferencial de inflación con la Unión Europea en el 1,1 es decir 0,2 puntos más que en abril; el dato de paro registrado del mes de mayo es el peor desde 1979; el déficit exterior aumentó en el primer trimestre el 19,1 y hoy es el 11,3 del PIB. Hoy dice la OCDE en sus previsiones que España crecerá el 1,6 este año, menos de la media de la Unión Europea, y hay muchos más datos en estos siete días: sobre ventas minoristas, ventas de automóviles, constituciones de hipotecas, el sector turístico. No voy a entrar en ellos porque no hay tiempo, pero le diré una cosa: la confianza de las empresas y de las familias españolas está en